In [172]:
import pandas as pd
import numpy as np
from glob import glob
import talib

In [176]:
def cci(sell_fee=0.0004, buy_fee=0.0004):
    
    all_result = pd.DataFrame()
    
    path = r'each_coin' 
    all_files = glob(path + "/*.csv")
    all_files.sort()
    for filename in all_files:
        coin_df = pd.read_csv(filename)
        
        coin_df['return'] = coin_df['open'].pct_change()
        coin_df['CCI'] = talib.CCI(coin_df['high'], coin_df['low'], coin_df['close'])
        

        trading_period = []
        trading_return = []
        win_trad = []
        trading_day = 0

        l_position = False
        s_position = False

        sell_fee = sell_fee
        buy_fee = buy_fee


        for i, row in coin_df.iterrows():
            date, price, returns, cci = pd.to_datetime(row['date']), row['open'], row['return'], row['CCI']

            if (l_position == True or s_position == True):  ## 포지션을 가지고 있을 경우 수익률 계산
                if l_position == True:
                    trading_return.append(returns)

                elif s_position == True:
                    trading_return.append(-returns)

            elif (l_position == False and s_position == False):
                trading_return.append(0)



            if cci > 100:  ## 과매수 시그널
                if (l_position == False and s_position == False):  ## 포지션이 아무것도 없었을 경우엔 숏 포지션 전환
                    s_position = True
                    trading_day = date  ## 포지션 시작 날짜

                elif l_position == True:  ## 롱 포지션이었을 경우 매도
                    l_position = False
                    trading_period.append((date-trading_day).days)  ## 포지션 기간 기록

                elif s_position == True:
                    pass


            elif cci < -100:  ## 과매도 시그널
                if (l_position == False and s_position == False): ## 포지션이 아무것도 없었을 경우 롱 포지션
                    l_position = True
                    trading_day = date

                elif s_position == True:  ## 숏포지션이었을 경우 환매
                    s_position = False
                    trading_period.append((date-trading_day).days)

                elif l_position == True:
                    pass

            else:
                pass


        val = 1
#         mr = 0  
#         mdd = 0  
        hr = 0  ## 각 포지션들 중에서 가장 높은 수익률
        lr = 0  ## 각 포지션들 중에서 가장 낮은 수익률
        for i in trading_return:
            if i == 0:
                if val > 1:
                    win_trad.append(1)
                    if val > (hr+1):
                        hr = val-1
                elif val < 1:
                    win_trad.append(0)
                    if val < (lr+1):
                        lr = val-1
                val = 1
            else:
                val *= (1+i)



        result = pd.DataFrame({'Whole Period': len(coin_df),
                              'Whole Return(%)': (coin_df['open'].iloc[-1]-coin_df['open'].iloc[0])/coin_df['open'].iloc[0] * 100,
                              'Whole Annual Return(%)': (coin_df['open'].iloc[-1]-coin_df['open'].iloc[0])/coin_df['open'].iloc[0] / len(coin_df) * 365 * 100,
                              'Whole Annual Vol': np.std(trading_return)*np.sqrt(365),
                              'Trading Period': sum(trading_period),
                              'Trading Return(%)': (np.prod(list(map(lambda x: x+1, trading_return)))-1) * 100,
                              'Trading Ave Period': sum(trading_period)/len(win_trad),
                              'Trading High Return(%)': hr * 100,
                              'Trading Low Return(%)': lr * 100,
                              '# Trades': len(win_trad),
                              'Winning Rate(%)': sum(win_trad)/len(win_trad) * 100,
                              'Trading Annual Return(%)': (np.prod(list(map(lambda x: x+1, trading_return)))-1)*(365/len(coin_df)) * 100,
                              'B&H Sharpe': coin_df['return'].mean()/coin_df['return'].std()*np.sqrt(365),
                              'Trading Sharpe': np.mean(trading_return)/np.std(trading_return)*np.sqrt(365)}, index=[filename.strip('each_coin/, .csv')])
        
        all_result = pd.concat([all_result, result])
        


    return all_result.round(decimals=3)

In [177]:
result = cci()

In [178]:
result

,Whole Period,Whole Return(%),Whole Annual Return(%),Whole Annual Vol,Trading Period,Trading Return(%),Trading Ave Period,Trading High Return(%),Trading Low Return(%),# Trades,Winning Rate(%),Trading Annual Return(%),B&H Sharpe,Trading Sharpe
1INCH,187,-12.029,-23.478,1.698,122,-91.633,15.250,18.356,-85.389,8,37.500,-178.856,0.974,-1.905
AAVE,257,459.342,652.373,1.601,188,-95.786,17.091,18.431,-87.536,11,54.545,-136.038,2.285,-1.976
ADA,516,2337.798,1653.675,1.254,436,-99.638,22.947,21.991,-93.157,19,47.368,-70.480,2.371,-2.482
ADA_coinmarketcap,1392,5388.930,1413.046,1.486,1253,-100.000,19.889,16.618,-111.210,63,39.683,-26.221,1.342,-2.556
ALGO,379,261.522,251.862,1.391,321,-96.689,17.833,20.822,-83.567,18,44.444,-93.117,1.569,-1.603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YFII,300,-65.148,-79.263,1.441,257,-86.355,19.769,16.927,-57.276,13,38.462,-105.065,0.098,-0.953
ZEC,511,81.207,58.005,1.265,422,-98.560,18.348,24.715,-47.691,23,30.435,-70.400,1.008,-1.702
ZEN,218,369.643,618.897,1.897,144,-98.082,28.800,11.268,-88.993,5,20.000,-164.220,2.267,-2.452
ZIL,377,268.416,259.872,1.548,303,-98.324,20.200,16.284,-84.641,15,40.000,-95.194,1.569,-1.703


In [175]:
result.to_csv('cci_100_result.csv')